In [1]:
import pandas as pd
from ptrail.core.TrajectoryDF import PTRAILDataFrame
from ptrail.features.kinematic_features import KinematicFeatures

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
from src.utils.test_utils import TestUtils
from src.selection.select import Selection
from src.utils.general_utils import Utilities
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
df = pd.read_csv('../datasets/fishes.csv', engine='python')
fishes_dataset = PTRAILDataFrame(data_set=df,
                                 traj_id='traj_id',
                                 datetime='DateTime',
                                 latitude='lat',
                                 longitude='lon')
ready_dataset = KinematicFeatures.create_distance_column(fishes_dataset)
ready_dataset = ready_dataset.loc[ready_dataset['Species'].isin([1, 2])]
ready_dataset

lat        lon  Species  Distance
traj_id DateTime                                                 
1058    2013-01-04 05:05:00 -5.2568  71.657683        1       NaN
        2013-01-05 18:27:00 -5.2568  71.657683        1       0.0
        2013-01-05 18:29:00 -5.2568  71.657683        1       0.0
        2013-01-05 18:33:00 -5.2568  71.657683        1       0.0
        2013-01-06 04:52:00 -5.2568  71.657683        1       0.0
...                             ...        ...      ...       ...
1654    2016-04-04 12:30:00 -5.2499  71.660270        1       0.0
        2016-04-04 12:31:00 -5.2499  71.660270        1       0.0
        2016-05-04 01:50:00 -5.2499  71.660270        1       0.0
        2016-05-04 01:51:00 -5.2499  71.660270        1       0.0
1674    2016-04-18 11:13:00  0.0000   0.000000        2       NaN

[356790 rows x 4 columns]

In [3]:
seed_generator = Utilities.generate_pi_seed(20)
seed_vals = [next(seed_generator) for i in range(20)]
shake_percentages = [0.2, 0.4, 0.6]
circle_methods = ['on', 'in']
ml_models = [ExtraTreesClassifier(), GradientBoostingClassifier(), RandomForestClassifier()]
scaler = MinMaxScaler((0, 1))

distance_results = [["seed", "on_20%_dist", "on_20%_std", "on_40%_dist", "on_40%_std", "on_60%_dist", "on_60%_std",
                    "in_20%_dist","in_20%_std","in_40%_dist","in_40%_std","in_60%_dist","in_60%_std"]]

model_results = [["seed", "model", "baseline", "in_20%_f1", "in_40%_f1", "in_60%_f1", "on_20%_f1", "on_40%_f1", "on_60%_f1"]]

for seed in seed_vals:
    # Intermediate lists for storing distance and model score values.
    distance_row = [seed]

    # Set apart 20% data for testing that augmentation process will never see.
    train, test_x, test_y = TestUtils.get_test_train_data(dataset=ready_dataset, seed_val=seed,
                                                          class_col='Species', k=0.8)

    model_row = TestUtils.create_model_row(seed, ml_models, "Species", train, test_x, test_y)
    for shake in shake_percentages:
        for method in circle_methods:
            # Randomly select 30% of trajectories to be augmented.
            selected = Selection.select_randomly(train, seed, k=0.3)

            # Augment the trajectories.
            train_x, train_y = TestUtils.augment_trajectories_using_random_strategy(dataset=train,
                                                                                    percent_to_shake=shake,
                                                                                    ids_to_augment=selected,
                                                                                    circle=method,
                                                                                    n_augmentations=5,
                                                                                    class_col="Species")
            mean, std = TestUtils.find_original_and_augmentation_pairs_and_calculate_differences(train_x, selected)
            distance_row.append(mean)
            distance_row.append(std)

            for i in range(len(ml_models)):
                f1_score = TestUtils.train_model_and_evaluate(ml_models[i], scaler.fit_transform(train_x), train_y,
                                                              scaler.fit_transform(test_x), test_y, seed)
                model_row[i].append(f1_score)

    model_results.extend(model_row)
    distance_results.append(distance_row)

    print(model_row)

[[1415, 'ExtraTreesClassifier', 0.439, 0.2375, 0.2375, 0.2375, 0.2375, 0.2375, 0.2375], [1415, 'GradientBoostingClassifier', 0.439, 0.2375, 0.2375, 0.2375, 0.2375, 0.2375, 0.2375], [1415, 'RandomForestClassifier', 0.439, 0.2375, 0.2375, 0.2375, 0.2375, 0.2375, 0.2375]]
[[9265, 'ExtraTreesClassifier', 0.0916, 0.6589, 0.6589, 0.6589, 0.6589, 0.6589, 0.6589], [9265, 'GradientBoostingClassifier', 0.0916, 0.6589, 0.6589, 0.6589, 0.6589, 0.6589, 0.6589], [9265, 'RandomForestClassifier', 0.0916, 0.6589, 0.6589, 0.6589, 0.6589, 0.6589, 0.6589]]
[[3589, 'ExtraTreesClassifier', 0.142, 0.142, 0.142, 0.142, 0.142, 0.142, 0.142], [3589, 'GradientBoostingClassifier', 0.142, 0.142, 0.142, 0.142, 0.142, 0.142, 0.142], [3589, 'RandomForestClassifier', 0.142, 0.142, 0.142, 0.142, 0.142, 0.142, 0.142]]
[[7932, 'ExtraTreesClassifier', 0.439, 0.439, 0.439, 0.439, 0.439, 0.439, 0.439], [7932, 'GradientBoostingClassifier', 0.439, 0.439, 0.439, 0.439, 0.439, 0.439, 0.439], [7932, 'RandomForestClassifier', 0.4

In [4]:
import csv

file_path = "./results/fishes_distances.csv"
with open(file_path, mode="w") as file:
    writer = csv.writer(file)
    for item in distance_results:
        writer.writerow(item)
    print(f"File successfully written to: {file_path}")

file_path = "./results/fishes_f1_score.csv"
with open(file_path, mode="w") as file:
    writer = csv.writer(file)
    for item in model_results:
        writer.writerow(item)
    print(f"File successfully written to: {file_path}")

File successfully written to: ./results/fishes_distances.csv
File successfully written to: ./results/fishes_f1_score.csv
